# Query the Nemoretriever OCR v1 API

This notebook demonstrates how to interact with the NVIDIA Nemoretriever OCR v1 NIM container using Python. 
It covers launching the NIM container and sending an inference request with a base64 encoded image.

Reference: [NVIDIA Build - Nemoretriever OCR](https://build.nvidia.com/nvidia/nemoretriever-ocr-v1/deploy)

## 1. Launch NIM

Before running the Python code, you need to launch the Nemoretriever OCR NIM container. 
Run the following command in your terminal:

```bash
export NGC_API_KEY=<PASTE_API_KEY_HERE> 
export LOCAL_NIM_CACHE=~/.cache/nim 
mkdir -p "$LOCAL_NIM_CACHE" 

# Start the OCR NIM 
docker run -it --rm \
  --gpus all \
  --shm-size=16GB \
  -e NGC_API_KEY=$NGC_API_KEY \
  -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
  -u $(id -u) \
  -p 8000:8000 \
  nvcr.io/nim/nvidia/nemoretriever-ocr-v1:1.2.1
```

## 2. Install Dependencies

In [ ]:
!pip install requests pillow

## 3. Prepare Image

We will download an example image and encode it to base64.

In [ ]:
import base64
import requests
from io import BytesIO
from PIL import Image

# Example Image URL (using the same one as before for consistency)
image_url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"

# Download image
response = requests.get(image_url)
image_content = response.content

# Encode to base64
encoded_image = base64.b64encode(image_content).decode("utf-8")
print("Image encoded successfully.")

## 4. Query the API

Send the base64 encoded image to the local inference API.

In [ ]:
import requests
import json

url = "http://localhost:8000/v1/infer"
payload = {
    "input": [
        {
            "type": "image_url",
            "url": f"data:image/png;base64,{encoded_image}"
        }
    ]
}
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

try:
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    result = response.json()
    print(json.dumps(result, indent=2))
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
    print("Ensure the NIM container is running on port 8000.")